# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
%matplotlib inline

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config2 import gkey
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data = pd.read_csv("weather_data.csv")
city_data.head()

,Unnamed: 0,City,Cloudiness,Country,Datetime,Humidity,Lat,Long,Max Temp,Wind Speed
0,0,Avarua,20,CK,06/22/2021,61,-21.2078,-159.7750,80.65,5.75
1,1,Thompson,75,CA,06/22/2021,35,55.7435,-97.8558,69.80,9.22
2,2,Lyuban',34,BY,06/22/2021,94,52.7985,28.0048,65.73,7.63
3,3,Barrow,1,US,06/22/2021,65,71.2906,-156.7887,44.62,8.05
4,4,Berlevåg,100,NO,06/22/2021,90,70.8578,29.0864,44.13,19.35


In [4]:
cities = city_data.drop(columns=['Unnamed: 0'])

cities.head()

,City,Cloudiness,Country,Datetime,Humidity,Lat,Long,Max Temp,Wind Speed
0,Avarua,20,CK,06/22/2021,61,-21.2078,-159.7750,80.65,5.75
1,Thompson,75,CA,06/22/2021,35,55.7435,-97.8558,69.80,9.22
2,Lyuban',34,BY,06/22/2021,94,52.7985,28.0048,65.73,7.63
3,Barrow,1,US,06/22/2021,65,71.2906,-156.7887,44.62,8.05
4,Berlevåg,100,NO,06/22/2021,90,70.8578,29.0864,44.13,19.35


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
!jupyter nbextension enable --py gmaps

!jupyter nbextension enable --py widgetsnbextension

!pip install npm

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [6]:
humidity = cities["Humidity"].astype(float)
maxhum = humidity.max()
locations = cities[['Lat', 'Long']]

In [7]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhum,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
n_city_df = cities.loc[(cities["Max Temp"] > 70) & (cities["Max Temp"] < 80) & (cities["Cloudiness"] == 0), :]
n_city_df = n_city_df.dropna(how='any')
n_city_df.reset_index(inplace=True)
del n_city_df['index']
n_city_df.head()

,City,Cloudiness,Country,Datetime,Humidity,Lat,Long,Max Temp,Wind Speed
0,Faya,0,SA,06/22/2021,13,18.3851,42.4509,70.32,8.57
1,Murgab,0,TM,06/22/2021,43,37.4966,61.9714,73.27,6.91
2,Zhengjiatun,0,CN,06/22/2021,43,43.5064,123.5064,74.95,6.24
3,Hartselle,0,US,06/22/2021,73,34.4434,-86.9353,78.04,6.08
4,Nikolskoye,0,RU,06/22/2021,73,59.7035,30.7861,73.40,6.71


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotels = []

for i in range(len(n_city_df)):
    lat = n_city_df.loc[i]['Lat']
    lng = n_city_df.loc[i]['Long']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
n_city_df["Hotel Name"] = hotels
hotel_df = n_city_df.dropna(how='any')
hotel_df.head()

,City,Cloudiness,Country,Datetime,Humidity,Lat,Long,Max Temp,Wind Speed,Hotel Name
0,Faya,0,SA,06/22/2021,13,18.3851,42.4509,70.32,8.57,الغال
1,Murgab,0,TM,06/22/2021,43,37.4966,61.9714,73.27,6.91,Mary
2,Zhengjiatun,0,CN,06/22/2021,43,43.5064,123.5064,74.95,6.24,Siping
3,Hartselle,0,US,06/22/2021,73,34.4434,-86.9353,78.04,6.08,Hartselle
4,Nikolskoye,0,RU,06/22/2021,73,59.7035,30.7861,73.40,6.71,Saint Petersburg


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)


# Display figure

fig

Figure(layout=FigureLayout(height='420px'))